### TODO: add to the requirements.txt

!pip install tweepy==3.10.0 

!pip install requests 

!pip install twython==3.9.1

In [1]:
import json
from twython import Twython, TwythonError, TwythonRateLimitError
import os
import pandas as pd

In [2]:
with open("../bearer_token.txt") as tokens:
    tokens_json = json.loads(tokens.read())
    #os.environ['BEARER_TOKEN'] = token.read()[:-1]
    os.environ['CONSUMER_KEY'] = tokens_json['consumer_key']
    os.environ['CONSUMER_SECRET'] = tokens_json['consumer_secret']
    os.environ['ACESS_TOKEN'] = tokens_json['access_token']
    os.environ['ACESS_TOKEN_SECRET'] = tokens_json['access_token_secret']

In [3]:
consumer_key = os.environ.get("CONSUMER_KEY")
consumer_secret = os.environ.get("CONSUMER_SECRET")
access_token = os.environ.get("ACCESS_TOKEN")
access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")

In [4]:
twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret) 

In [5]:
def timeline(user_id, max_id=None):
#       """ 
#   Loop over a user's timeline, starting at max_id. 
#   Generator.
#  
#   We can get up to 15 pages. 
#   This function loops up to 16 times to make the base case 
#   `len(tweets) == 0` trigger.
#   """
    for i in range(16):
        tweets = twitter.get_user_timeline(screen_name=user_id,
                                      max_id=max_id,
                                      count=200,
                                      trim_user=True,
                                      exclude_replies=True,
                                      include_rts=False)
        if len(tweets) > 0:  # last page should have zero results
            for tweet in tweets:
                max_id = tweet['id'] - 1
                yield tweet
        else:
            break

In [6]:
from datetime import datetime
import time

def sleep_until(ts):
 # """ Sleep until the given UTC UNIX TIMESTAMP. """
    next_time = datetime.utcfromtimestamp(int(ts))
    now = datetime.utcnow()
    offset = (next_time - now).seconds
    print("Enhancing calm. Next try: {} (Currently {})".format(next_time, now))
    print("Sleeping for {}...".format(offset))
    time.sleep(offset)
    print("Continuing...")

# OLD: PENDING DELETE

users_target = [{'user':'BernieSanders',                               
'party':'Democratic Party'         
},
{'user':'ewarren',                 
'party':'Democratic Party'         
},
{'user':'AOC',                     
'party':'Democratic Party'         
},
{'user':'CoryBooker',              
'party':'Democratic Party'         
},
{'user':'KamalaHarris',            
'party':'Democratic Party'         
},
{'user':'SpeakerPelosi',           
'party':'Democratic Party'         
},
{'user':'AlexPadilla4CA',          
'party':'Democratic Party'         
},
{'user':'DickDurbin',              
'party':'Democratic Party'         
},
{'user':'JeanneShaheen',                   
'party':'Democratic Party'                 
},
{'user':'JeffMerkley',                      
'party':'Democratic Party'                 
},
{'user':'JoeBiden',                         
'party':'Democratic Party'                 
},
{'user':'tedcruz',                                
'party':'Republican Party'
},
{'user':'marcorubio',
'party':'Republican Party'
},
{'user':'RandPaul',
'party':'Republican Party'
},
{'user':'MittRomney',
'party':'Republican Party'
},
{'user':'BillHagertyTN',
'party':'Republican Party'
},
{'user':'JohnCornyn',
'party':'Republican Party'
},
{'user':'MarshaBlackburn',
'party':'Republican Party'
},
{'user':'senrobportman',
'party':'Republican Party'
},
{'user':'DanCrenshawTX',  
'party':'Republican Party'
},
{'user':'SteveScalise',   
'party':'Republican Party'
},
{'user':'GOPLeader',      
'party':'Republican Party'
}]                      

In [7]:
list_members_congress_df = pd.read_csv('../data/eeuu_member_of_congress.csv', sep=';')
list_members_congress_df

,State,Member of Congress,Name,Party,Twitter,Instagram,Facebook
0,North Carolina 12th District,U.S. Representative,"Adams, Alma",D,@RepAdams,@repadams,x
1,Alabama 4th District,U.S. Representative,"Aderholt, Robert",R,@Robert_Aderholt,@robert_aderholt,x
2,California 31st District,U.S. Representative,"Aguilar, Pete",D,@RepPeteAguilar,@rep.peteaguilar,x
3,Georgia 12th District,U.S. Representative,"Allen, Rick",R,@RepRickAllen,@rep_rickallen,x
4,Texas 32nd District,U.S. Representative,"Allred, Colin",D,@RepColinAllred,@repcolinallred,x
...,...,...,...,...,...,...,...
536,Oregon,U.S. Senator,"Wyden, Ron",D,@RonWyden,@ronwyden,x
537,Kentucky 3rd District,U.S. Representative,"Yarmuth, John A.",D,@RepJohnYarmuth,NaN,x
538,Alaska At-Large,U.S. Representative,"Young, Don",R,@RepDonYoung,@repdonyoung,x
539,Indiana,U.S. Senator,"Young, Todd",R,@SenToddYoung,@sentoddyoung,x


In [8]:
list_members_congress_df = list_members_congress_df.dropna(subset = ['Party', 'Twitter'])
list_members_congress_df['Twitter'] = list_members_congress_df['Twitter'].apply(lambda x: x.replace('@', ''))

/tmp/ipykernel_3925/629263200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_members_congress_df['Twitter'] = list_members_congress_df['Twitter'].apply(lambda x: x.replace('@', ''))


In [17]:
fields = ['id', 'user', 'created_at', 'text', 'lang', 'label', 'retweet_count', 'favorite_count']
max_id = None
for index, row in list_members_congress_df.iterrows():
    df_output = pd.DataFrame(columns = fields)
    while True:
        try:
            name_user = row['Twitter']
            print(f'############# Scraping {name_user} timeline #############')
            for idx, tweet in enumerate(timeline(name_user, max_id)):
                if (idx+1) % 1000 == 0:
                    print(f'Scanned {idx+1} tweets')
                sub = {k: v for k, v in tweet.items() if k in fields}
                sub['user'] = sub['user']['id']
                sub['label'] = row['Party']
                df_output = df_output.append(sub, ignore_index=True)
                max_id = sub['id']
            max_id = None
            break
        except TwythonRateLimitError as e:
            sleep_until(e.retry_after) # sleep until the given date
        except TwythonError as e:
            # I *think* this is caused by protected profiles. 
            # I can read some user profile info, but not the timeline
            print(e)
            print("Skipping...")
            break
    df_output.to_csv(f'../data/{name_user}.csv', sep='|', encoding='utf-8', index=False)

############# Scraping RepRonKind timeline #############
Scanned 1000 tweets


----------------------------------------------------------------------------------------------------------